In [1]:
import numpy as np
import gym
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import time
import matplotlib.pyplot as plt

In [2]:
# Create gym environment
env = gym.make('Pendulum-v0')
env.reset()
#action space is a Box(1,) with values between [-2,2], joint effort
print(env.action_space.low)
#observation space is 3d angle of pendulum cos, sin, velocity max:1,1,8; min:-1,-1,8
print(env.observation_space.low)

[-2.]
[-1. -1. -8.]


In [3]:
#reward formular: -(theta^2 + 0.1*theta_dt^2 + 0.001*action^2) (-16.27 is worst, 0 best)
print(env.reset())
a = env.action_space.sample()
print(a)
state, reward, done, info = env.step(a)
print(state, reward, done, info)

[0.99952834 0.03070997 0.98436374]
[0.19525401]
[0.99659476 0.0824553  1.03668432] -0.09787872067163456 False {}


In [9]:
from Discretization import Discretization

larry = Discretization("degree_only", "Pendulum",state_space_size=(18+1, 16+1), action_space_size=17)

print(larry.map_to_index(-90, 6))

[181, 14]


In [ ]:
"""
    Regression of old state and performed action to new state and observed reward.
"""

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Learning episodes / amount of samples for regression
epochs = 1000

rtx = []
rty = []
stx = []
sty = []
plotr = []
plots = []

regressorReward = RandomForestRegressor(n_estimators=10, min_samples_split=2)
regressorState = RandomForestRegressor(n_estimators=10, min_samples_split=2)

old_state = env.reset()

print("Regression...")
for i in range(epochs):
    
    action = env.action_space.sample()
    next_state, reward, done, info = env.step(action)

    rtx.append(np.append(old_state ,action))
    rty.append(reward)
    stx.append(np.append(old_state,action))
    sty.append(next_state)
    
    if i%50==0: # 50 works nicely
        
        regressorReward.fit(rtx, rty)
        fitrtx = regressorReward.predict(rtx)
        mse = mean_squared_error(rty, fitrtx)
        plotr.append(mse)

        
        regressorState.fit(stx, sty)
        fitstx = regressorState.predict(stx)
        mse = mean_squared_error(sty, fitstx)

        plots.append(mse)
    
    old_state = np.copy(next_state)

print("...done")
plt.figure(0)
plt.plot(plotr, label="Loss for reward fitting")

plt.plot(plots, label="Loss for state fitting")
plt.legend()
print(regressorReward)

In [ ]:
"""
   Value Iteration
   
"""

def value_iteration(disc, theta, gamma):
    
    print("Value iteration...")

    value_function = np.ones(shape=disc.state_space_size)
    policy = np.ones(shape=disc.state_space_size)
    
    delta = theta
    
    while delta >= theta:
        
        delta = 0
        
        for s0 in disc.state_space[0]:
            for s1 in disc.state_space[1]:
                # Get (only positive) indexes for (possibly negative) discrete state(s)
                index = disc.map_to_index([s0, s1])
                print(index)

                v = value_function[index[0], index[1]]

                # Iterate over all actions to get action maximizing expected reward
                amax = 2
                rmax = -100
                for a in disc.action_space:

                    for a in disc.action_space:
                        # Get sufficient state and reward from regressors
                        x = np.array([s0, s1, a])
                        x = x.reshape(1,-1)
                        next_s = regressorState.predict(x).T.reshape(-1,)
                        r = regressorReward.predict(x)

                        # Discretize sufficient state
                        next_index = disc.map_to_index([next_s[0], next_s[1]])
                        print(next_index)
                        # Calculate expected reward
                        # Deterministic case; we do not need probability distribution
                        expected_reward = r + gamma*value_function[next_index[0], next_index[1]]

                        if rmax < expected_reward:
                            amax = a
                            rmax = expected_reward 

                # Define value function by maximum expected reward per state
                value_function[index[0], index[1]] = rmax
                # Define policy by action achieving maximum expected reward per state
                policy[index[0], index[1]] = amax

        print("Delta: ", delta)
                    
                    
    print()
    print("...done")
    return value_function, policy

value_function, policy = value_iteration(disc=larry, theta=1.0, gamma=0.1)
                    

In [ ]:
"""
    Policy Iteration
    
    Gives convergence towards the optimal policy by iteratively
    performing Policy Evaluation and Policy Improvement

"""

def policy_iteration(disc, theta, gamma):        
    
    print("Policy iteration...")

    value_function = np.ones(shape=disc.state_space_size)
    policy = np.zeros(shape=disc.state_space_size)

    
    def policy_evaluation(theta, gamma):
        print()
        print("Evaluating policy")
        delta = theta
        while delta >= theta:
            delta = 0
            # Iteratate over discrete state space
            for s0 in disc.state_space[0]:
                for s1 in disc.state_space[1]:
                    for s2 in disc.state_space[2]:
                        
                        # Get index for state 
                        # The method already iterates over a discretized state space
                        # But the states need to get mapped to a positive index do to possible 'negative' states
                        index = disc.map_to_index([s0, s1, s2])
                        
                        v = value_function[index[0], index[1], index[2]]
                        
                        """
                         V(s) = Sum...p(s',r|s,pi(s))[r+gamma*V(s')]
                         
                        """
                        a = policy[index[0], index[1], index[2]]
                        
                        # input for regression
                        x = np.array([s0, s1, s2, a]).reshape(1, -1)
                        
                        # Predict next state and reward with regressors
                        next_s = regressorState.predict(x).T.reshape(-1,)
                        r = regressorReward.predict(x)      
                        
                        next_index = disc.map_to_index([next_s[0], next_s[1], next_s[2]])
                                          
                        value_function[index[0], index[1], index[2]] = r + gamma*value_function[next_index[0],
                                                                              next_index[1], next_index[2]]
                                          
                        delta = max(delta, v - value_function[index[0], index[1], index[2]])
            print("Delta: ", delta)
    
    
    def policy_improvement(gamma):
        print()
        print("Improving policy")
        policy_stable = True
        for s0 in disc.state_space[0]:
                for s1 in disc.state_space[1]:
                    for s2 in disc.state_space[2]:
                        
                        # Indexing
                        index = disc.map_to_index([s0, s1, s2])
                        
                        old_action = policy[index[0], index[1], index[2]]
                        
                        """
                            pi(s) = argmax_a ... 
                            We do not have to care about the prob. distribution,
                            as we have a deterministic env.
                            
                        """
                        # Iterate over all actions and get the one with max. expected reward
                        amax = 2
                        rmax = -100
                        for a in disc.action_space:
                            x = np.array([s0, s1, s2, a])
                            x = x.reshape(1,-1)
                            next_s = regressorState.predict(x).T.reshape(-1,)
                            next_index = disc.map_to_index([next_s[0], next_s[1], next_s[2]])
                            r = regressorReward.predict(x)
                            expected_reward = r + gamma*value_function[next_index[0], next_index[1], next_index[2]]
                            if rmax < expected_reward:
                                amax = a
                                rmax = expected_reward 
                        policy[index[0], index[1], index[2]] = amax # TODO
                        
                        if old_action != policy[index[0], index[1], index[2]]:
                            policy_stable = False
                            
        print("Policy stable: ", policy_stable)
        return policy_stable
        
    # Run until policy is stable
    stable_policy = False
    while not stable_policy:
        policy_evaluation(theta, gamma)
        stable_policy = policy_improvement(gamma)
    
    print()
    print("...done")
    return value_function, policy
    
# value_function, policy = policy_iteration(larry, theta=1, gamma=0.1)


In [ ]:
state = env.reset()

for _ in range(5):
    print("Current state", larry.map_to_index(state))
    
    a = env.action_space.sample()
    print("Action: ", a)
    
    # input for regression
    x = np.array([state[0], state[1], state[2], a]).reshape(1, -1)
    
    # Do regression
    print("Predicted reward: ", regressorReward.predict(x))
    state = regressorState.predict(x).T.reshape(-1,)
    print("Predicted next state: ",  larry.map_to_index(state))                

    
    # Perform action
    state, reward, done, info = env.step(a)
    print("True reward: ", reward)
    print("True next state: ", larry.map_to_index(state))
    print()
    

In [ ]:
"""
    Some stuff to see the predictions, discretizations and learned functions in action

"""
rewards_per_episode = []

for e in range(5):
    
    # Discretize first state
    index = larry.map_to_index(env.reset())
    
    rewards_per_timestep = []
    
    for t in range(1000):
        # Render environment
        # env.render()

        # Do step according to policy and get observation and reward
        action = np.array([policy[index[0], index[1], index[2]]])
        print(action)
        #action = env.action_space.sample()
        observation, reward, done, info = env.step(action)

        rewards_per_timestep.append(reward)
        
        # Discretize observed state
        index = larry.map_to_index(observation)

        if done:
            print("Episode {} finished after {} timesteps".format(e+1, t+1))
            break
    
    rewards_per_episode.append(rewards_per_timestep)

# Average reward over episodes
rewards = np.average(rewards_per_episode, axis=0)
print(np.shape(rewards))
        
env.close()

# Plot rewards per timestep averaged over episodes
plt.figure()
plt.plot(rewards, label='Average reward per timestep')
plt.legend()
plt.show()